## Homework Assignment

A 3800 MWth rated BWR wishes to operate on an equilibrium 18-month cycle with an expected capacity factor (including refueling outage) of 95%. The total core loading consists of 800 fuel assemblies containing 180 MTU. The fuel region average discharge burnup limit is 44,000 MWD/MTU, and it is determined that a four-region core should be employed.

Determine the enrichment of feed fuel assemblies for each of the following approximations to the radial power distribution.

a) Flat power distribution.

b) Uneven power distribution, where the power distribution is given in the following table.

| Number of time batch burned at BOC | Avg. relative power of batch (batch power / avg. power over core) |
|------------------------------------|-------------------------------------------------------------------|
| 0                                  | 0.80                                                              |
| 1                                  | 1.30                                                              |
| 2                                  | 1.20                                                              |
| 3                                  | 0.70                                                              |
---

The following table provides reactivity as a function of burnup (BU) and enrichment. Interpolate or extrapolate as needed to determine the batch reactivity as a function of feed enrichment. Utilize a non-leakage probability L = 0.96 and target k_EOC = 1.005.

| BU (GWD/MTU) \ Enrichment (%) | 2.1  | 2.6  | 3.1  | 3.6  | 4.1  | 4.6  |
|-------------------------------|-------|-------|-------|-------|-------|-------|
| 0                             | 1.120 | 1.170 | 1.220 | 1.270 | 1.320 | 1.370 |
| 10                            | 1.000 | 1.050 | 1.100 | 1.150 | 1.200 | 1.250 |
| 20                            | 0.900 | 0.950 | 1.000 | 1.050 | 1.100 | 1.150 |
| 30                            | 0.840 | 0.890 | 0.940 | 0.990 | 1.040 | 1.090 |
| 40                            | 0.790 | 0.840 | 0.890 | 0.940 | 0.990 | 1.040 |
| 50                            | 0.720 | 0.773 | 0.825 | 0.878 | 0.930 | 0.983 |
| 60                            | 0.640 | 0.690 | 0.740 | 0.790 | 0.840 | 0.890 |


In [2]:
from IPython.display import display, Math
import math
import numpy as np

# Defining the given data
thermal_power = 3800  # in MWth
refueling_cycle = 18  # in months
capacity_factor = 0.95  # in percent
total_core_loading = 800  # number of fuel assemblies
core_mass = 180  # in MTU
burnup_limit = 44000  # in MWD/MTU
num_regions = 4  # number of radial regions

# Radial power distribution for uneven power distribution
power_distribution = {
    0: 0.80,
    1: 1.30,
    2: 1.20,
    3: 0.70
}

# Burnup and enrichment data from the table
burnup_values = [0, 10, 20, 30, 40, 50, 60]  # BU (GWD/MTU)
enrichment_levels = [2.1, 2.6, 3.1, 3.6, 4.1, 4.6]  # Enrichment (%)

reactivity_table = np.array([
    [1.120, 1.170, 1.220, 1.270, 1.320, 1.370],
    [1.000, 1.050, 1.100, 1.150, 1.200, 1.250],
    [0.900, 0.950, 1.000, 1.050, 1.100, 1.150],
    [0.840, 0.890, 0.940, 0.990, 1.040, 1.090],
    [0.790, 0.840, 0.890, 0.940, 0.990, 1.040],
    [0.720, 0.773, 0.825, 0.878, 0.930, 0.983],
    [0.640, 0.690, 0.740, 0.790, 0.840, 0.890],
])

# Interpolation function for reactivity based on BU and enrichment
def interpolate_reactivity(burnup, enrichment):
    # Ensure burnup and enrichment are within bounds of the table
    if burnup < min(burnup_values) or burnup > max(burnup_values):
        raise ValueError("Burnup value is out of bounds for interpolation.")
    if enrichment < min(enrichment_levels) or enrichment > max(enrichment_levels):
        raise ValueError("Enrichment value is out of bounds for interpolation.")

    # Interpolate in two steps: burnup first, then enrichment
    burnup_index = np.searchsorted(burnup_values, burnup) - 1
    enrichment_index = np.searchsorted(enrichment_levels, enrichment) - 1

    # Extract surrounding points for burnup and enrichment
    bu1, bu2 = burnup_values[burnup_index], burnup_values[burnup_index + 1]
    en1, en2 = enrichment_levels[enrichment_index], enrichment_levels[enrichment_index + 1]

    r1 = reactivity_table[burnup_index, enrichment_index]
    r2 = reactivity_table[burnup_index + 1, enrichment_index]
    r3 = reactivity_table[burnup_index, enrichment_index + 1]
    r4 = reactivity_table[burnup_index + 1, enrichment_index + 1]

    # Interpolate
    r_bu = r1 + (r2 - r1) * (burnup - bu1) / (bu2 - bu1)
    r_en = r3 + (r4 - r3) * (burnup - bu1) / (bu2 - bu1)
    return r_bu + (r_en - r_bu) * (enrichment - en1) / (en2 - en1)

# Example usage
try:
    reactivity = interpolate_reactivity(25, 3.0)  # Example: burnup=25, enrichment=3.0
    print(f"Interpolated reactivity: {reactivity:.3f}")
except ValueError as e:
    print(e)

Interpolated reactivity: 0.960


In [3]:
# Total energy production
Tot_energy = capacity_factor * thermal_power * refueling_cycle * 30

# Cycle burnup
Bu_cycle = Tot_energy / core_mass
display(f"Cycle burnup: {Bu_cycle:.2f} MWd/tU")

'Cycle burnup: 10830.00 MWd/tU'

## Cycle energy production
$$
\rho (Bu) = \frac{1}{N} \sum _i \rho _0 \left(  1 - (i-1) \frac{BU_{cycle}}{BU_{1}} - \frac{Bu}{BU_{1}} \right)
$$

Where $BU_1$ is the burnup of the core if it was just 1 region and can be computed from
$$
Bu_{cycle} = \frac{2}{N+1} BU_1
$$

In [4]:
# 1-region equivalent core Burnup
Bu_1 = Bu_cycle * (num_regions + 1) / 2
print(f"1-region equivalent core burnup: {Bu_1:.2f} MWd/tU")

1-region equivalent core burnup: 27075.00 MWd/tU


In [5]:
# We have a numerical correlation to evaluate the initial enrichment of the fuel
def calculate_xp(Bu1):
    """
    Calculate xp based on the given formula.
    
    Parameters:
    BUd (float): Value of BUd
    Nr (int): Value of Nr
    
    Returns:
    float: The calculated xp
    """
    # Constants
    a = 0.41201
    b = 0.11508
    c = 0.00023937

    # Calculate xp
    xp = a + b * Bu1 + c * (Bu1 ** 2)
    return xp

enrichment = calculate_xp(Bu_1/1000) # Convert Bu_1 to GWd/tU
print(f"Initial enrichment of the fuel: {enrichment:.2f} %")

Initial enrichment of the fuel: 3.70 %


In [6]:
Bu_region_0 = Bu_1 * 0.8
Bu_region_1 = Bu_1 * 1.3
Bu_region_2 = Bu_1 * 1.2
Bu_region_3 = Bu_1 * 0.7

enrichment_0 = calculate_xp(Bu_region_0/1000) # Convert Bu_region_0 to GWd/tU
enrichment_1 = calculate_xp(Bu_region_1/1000) # Convert Bu_region_1 to GWd/tU
enrichment_2 = calculate_xp(Bu_region_2/1000) # Convert Bu_region_2 to GWd/tU
enrichment_3 = calculate_xp(Bu_region_3/1000) # Convert Bu_region_3 to GWd/tU

print(f"Initial enrichment of the fuel for region 0: {enrichment_0:.2f} %")
print(f"Initial enrichment of the fuel for region 1: {enrichment_1:.2f} %")
print(f"Initial enrichment of the fuel for region 2: {enrichment_2:.2f} %")
print(f"Initial enrichment of the fuel for region 3: {enrichment_3:.2f} %")


Initial enrichment of the fuel for region 0: 3.02 %
Initial enrichment of the fuel for region 1: 4.76 %
Initial enrichment of the fuel for region 2: 4.40 %
Initial enrichment of the fuel for region 3: 2.68 %
